In [ ]:
#importing general plots
import numpy as np
import pandas as pd

In [ ]:
#taking out data from train.csv
dataset = pd.read_csv('train.csv')

In [ ]:
#Checking out dataset
dataset.head()

In [ ]:
#Shape of dataset
dataset.shape

In [ ]:
#Cleaning the texts
import re
import nltk

#Downloading stopwords
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
#Removing all the numbers and characters other than a to z and A to Z
review = re.sub('[^a-zA-Z]' , ' ' , dataset['comment_text'][0])
print(review)

In [ ]:
# for i in range(0 , 95851):
#     #Removing all the numbers and characters other than a to z and A to Z
#     review = re.sub('[^a-zA-Z]' , ' ' , dataset['comment_text'][i])
#     #Converting into lower case
#     review = review.lower()
#     #Splitting
#     review = review.split()
#     #Appying Stemmer 
#     ps = PorterStemmer()
#     review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
#     # Again join with space to form sentence
#     review = ' '.join(review)
#     #Appending to corpus list
#     #corpus.append(review)
#     dataset['comment_text'][i] = review

In [ ]:
dataset['comment_text'] = dataset['comment_text'].apply(lambda x: x.lower())
dataset['comment_text'] = dataset['comment_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [ ]:
dataset.head()   

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [ ]:
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(dataset['comment_text'].values)
X = tokenizer.texts_to_sequences(dataset['comment_text'].values)

In [ ]:
X = pad_sequences(X)

In [ ]:
X

In [ ]:
len(X)

In [ ]:
tokenizer.document_count

In [ ]:
X.shape

In [ ]:
embed_dim = 128
lstm_out = 196
max_features = 2000
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y = dataset.iloc[ : , [2,3,4,5,6,7]].values

In [ ]:
Y

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 100, batch_size=batch_size, verbose = 2)
model.save_weights("Weights.hdf5")

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

## New Post

In [1]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [64]:
def ingest():
    data = pd.read_csv('./train.csv')
    data.drop(['id'], axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)    
    return data

def testInjest():
    data_test = pd.read_csv('./test.csv')
    data_test.drop(['id'], axis=1, inplace=True)
    return data_test

data = ingest()
data_test = testInjest()
data.head(5)
data.shape
data['comment_text'] = data['comment_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['comment_text'] = data['comment_text'].apply(lambda x: x.lower())

data_test['comment_text'] = data_test['comment_text'].apply((lambda x: re.sub('[^a-zA-z]',' ',str(x))))
data_test['comment_text'] = data_test['comment_text'].apply(lambda x: x.lower())

dataset loaded with shape (95851, 7)


In [65]:
data_test.head(10)

,comment_text
0,orphaned non free media image cd jboevl ...
1,kentuckiana is colloquial even though the ...
2,hello fellow wikipedians i have just modified...
3,akc suspensions the morning call feb ...
4,[wiki_link talk celts]
5,hello xdes [wiki_link wikipedia introduction...
6,hi there i m i just wanted to let you know ...
7,i don t think a stuffed arm really counts as...
8,was it ever really a single in the uk
9,thanks for uploading [wiki_link image cloudco...


In [66]:
data_test.shape

(226998, 1)

In [67]:
def tokenize(tweet):
    try:
#        tweet = unicode(tweet.decode('utf-8').lower())
        tokens = tokenizer.tokenize(tweet)
#         tokens = filter(lambda t: not t.startswith('@'), tokens)
#         tokens = filter(lambda t: not t.startswith('#'), tokens)
#         tokens = filter(lambda t: not t.startswith('http'), tokens)
        return tokens
    except:
        return 'NC'

In [68]:
def postprocess(data, n=90000):
    data = data.head(n)
    data['comment_text'] = data['comment_text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
#    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

def postprocess1(data, n=226998):
    data = data.head(n)
    data['comment_text'] = data['comment_text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
#    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data


data = postprocess(data)
data_test = postprocess1(data_test)

progress-bar: 100%|██████████| 226998/226998 [01:07<00:00, 3368.75it/s]


In [69]:
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"[nonsense, kiss, off, geek, what, i, said, is,...",1,0,0,0,0,0
1,"[please, do, not, vandalize, pages, as, you, d...",0,0,0,0,0,0
2,"[points, of, interest, i, removed, the, points...",0,0,0,0,0,0
3,"[asking, some, his, nationality, is, a, racial...",0,0,0,0,0,0
4,"[the, reader, here, is, not, going, by, my, sa...",0,0,0,0,0,0


In [70]:
data_test.head()
data_test.shape

(226998, 1)

In [71]:
n=90000
x = np.array(data.head(n).comment_text)
xpTest = np.array(data_test.comment_text)

In [81]:
x

array([ list(['nonsense', 'kiss', 'off', 'geek', 'what', 'i', 'said', 'is', 'true', 'ill', 'have', 'your', 'account', 'terminated']),
       list(['please', 'do', 'not', 'vandalize', 'pages', 'as', 'you', 'did', 'with', 'this', 'edit', 'to', 'w', 's', 'merwin', 'if', 'you', 'continue', 'to', 'do', 'so', 'you', 'will', 'be', 'blocked', 'from', 'editing']),
       list(['points', 'of', 'interest', 'i', 'removed', 'the', 'points', 'of', 'interest', 'section', 'you', 'added', 'because', 'it', 'seemed', 'kind', 'of', 'spammy', 'i', 'know', 'you', 'probably', 'didnt', 'mean', 'to', 'disobey', 'the', 'rules', 'but', 'generally', 'a', 'point', 'of', 'interest', 'tends', 'to', 'be', 'rather', 'touristy', 'and', 'quite', 'irrelevant', 'to', 'an', 'area', 'culture', 'thats', 'just', 'my', 'opinion', 'though', 'if', 'you', 'want', 'to', 'reply', 'just', 'put', 'your', 'reply', 'here', 'and', 'add', 'talkbackjamiegraham', '08', 'on', 'my', 'talkpage']),
       ...,
       list(['history', 'of', 'th

In [73]:
len(xpTest)

226998

In [74]:
y = data.iloc[ : , [1,2,3,4,5,6]].values

In [75]:
y

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [77]:
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y, test_size=0.2)

In [82]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')
xpTest = labelizeTweets(xpTest, 'xpTEST')

72000it [00:00, 252579.52it/s]
18000it [00:00, 229859.86it/s]
226998it [00:02, 83107.69it/s] 


In [83]:
len(xpTest)

226998

In [84]:
n_dim = 200
tweet_w2v = Word2Vec(size=n_dim, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=10000, epochs=10)

100%|██████████| 72000/72000 [00:00<00:00, 1664287.10it/s]


34116603

In [85]:
tweet_w2v['good']

array([ 0.31690887,  0.09564571, -0.43064636, -0.19023147, -0.19107437,
       -0.18558665,  0.65920138, -0.35802716, -0.02900002, -0.82409728,
       -0.74998695, -0.69745839,  0.70544082,  1.46459186, -0.00538704,
       -1.19610989,  0.44915837,  0.61099321, -0.27231115, -0.70463902,
       -0.5501985 , -0.03105429, -0.01823857, -0.77969551, -0.29123259,
       -0.76161528, -0.5721913 ,  0.44861951, -0.89943337, -1.25175822,
       -0.11477406,  0.19114852, -0.1166639 ,  0.33315185,  0.18687882,
        1.95786095, -0.50527388,  0.06508212, -1.41979373,  1.05057168,
        0.28984976,  0.78972727,  0.25704905,  0.82874626,  1.01396799,
        0.79089791,  0.16578628, -0.74659199, -0.91490722, -0.50939864,
        0.23889644,  0.65001953, -0.25641647, -0.26716796,  0.57625043,
       -1.43881524, -0.96519917,  0.02352213,  0.53389335, -0.16220026,
        0.34284717,  0.87444288,  0.57828873, -1.07235849, -0.30634731,
       -0.29580462,  0.05337659, -0.61276406, -0.3762905 , -1.06

In [86]:
tweet_w2v.most_similar('good')

[('bad', 0.7510071992874146),
 ('nice', 0.7141897678375244),
 ('taking', 0.6639406085014343),
 ('great', 0.6470977067947388),
 ('hard', 0.6450170278549194),
 ('serious', 0.6316549181938171),
 ('reasonable', 0.6301519274711609),
 ('wiki', 0.6293802261352539),
 ('little', 0.6019768714904785),
 ('fine', 0.5994066596031189)]

In [87]:
# importing bokeh library for interactive dataviz
# import bokeh.plotting as bp
# from bokeh.models import HoverTool, BoxSelectTool
# from bokeh.plotting import figure, show, output_notebook

In [88]:
# defining the chart
# output_notebook()
# plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
#     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
#     x_axis_type=None, y_axis_type=None, min_border=1)

# # getting a list of word vectors. limit to 10000. each is of 200 dimensions
# word_vectors = [tweet_w2v[w] for w in list(tweet_w2v.wv.vocab.keys())[:5000]]

# # dimensionality reduction. converting the vectors to 2d vectors
# from sklearn.manifold import TSNE
# tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
# tsne_w2v = tsne_model.fit_transform(word_vectors)

# # putting everything in a dataframe
# tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
# tsne_df['words'] = list(tweet_w2v.wv.vocab.keys())[:5000]

# # plotting. the corresponding word appears when you hover on the data point.
# plot_tfidf.scatter(x='x', y='y', source=tsne_df)
# hover = plot_tfidf.select(dict(type=HoverTool))
# hover.tooltips={"word": "@words"}
# show(plot_tfidf)

In [89]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 14134


In [90]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [91]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

Finaltest_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, xpTest))])
Finaltest_vecs_w2v = scale(Finaltest_vecs_w2v)

72000it [00:40, 1763.67it/s]
18000it [00:11, 1600.68it/s]
226998it [02:23, 1587.04it/s]


In [94]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [95]:
from keras.layers import Dropout

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dropout(0.2))
model.add(Dense(103, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(103, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(103, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=100, batch_size=32)
model.save_weights("model.h5")
print("Saved model to disk")

Epoch 1/100
72000/72000 [==============================] - 9s 124us/step - loss: 0.3029 - acc: 0.9610
Epoch 2/100
72000/72000 [==============================] - 6s 85us/step - loss: 0.2967 - acc: 0.9900
Epoch 3/100
72000/72000 [==============================] - 6s 85us/step - loss: 0.2946 - acc: 0.9903
Epoch 4/100
72000/72000 [==============================] - 6s 84us/step - loss: 0.2936 - acc: 0.9904
Epoch 5/100
72000/72000 [==============================] - 6s 85us/step - loss: 0.2923 - acc: 0.9904
Epoch 6/100
72000/72000 [==============================] - 6s 88us/step - loss: 0.2920 - acc: 0.9903
Epoch 7/100
72000/72000 [==============================] - 7s 100us/step - loss: 0.2916 - acc: 0.9898
Epoch 8/100
72000/72000 [==============================] - 7s 92us/step - loss: 0.2910 - acc: 0.9901
Epoch 9/100
72000/72000 [==============================] - 6s 88us/step - loss: 0.2905 - acc: 0.9891
Epoch 10/100
72000/72000 [==============================] - 6s 88us/step - loss: 0.2908 -

72000/72000 [==============================] - 6s 77us/step - loss: 0.2856 - acc: 0.9817
Epoch 82/100
72000/72000 [==============================] - 6s 77us/step - loss: 0.2855 - acc: 0.9807
Epoch 83/100
72000/72000 [==============================] - 6s 77us/step - loss: 0.2858 - acc: 0.9798
Epoch 84/100
72000/72000 [==============================] - 6s 77us/step - loss: 0.2851 - acc: 0.9825
Epoch 85/100
72000/72000 [==============================] - 6s 86us/step - loss: 0.2850 - acc: 0.9811
Epoch 86/100
72000/72000 [==============================] - 6s 87us/step - loss: 0.2864 - acc: 0.9843
Epoch 87/100
72000/72000 [==============================] - 6s 88us/step - loss: 0.2855 - acc: 0.9816
Epoch 88/100
72000/72000 [==============================] - 6s 87us/step - loss: 0.2853 - acc: 0.9831
Epoch 89/100
72000/72000 [==============================] - 6s 88us/step - loss: 0.2853 - acc: 0.9860
Epoch 90/100
72000/72000 [==============================] - 6s 88us/step - loss: 0.2856 - acc: 

In [96]:
score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=2)
print (score[1])

0.99066666672


In [97]:
score

[0.29633516761991713, 0.99066666671964854]

In [98]:
predictions = model.predict(Finaltest_vecs_w2v, batch_size=128, verbose=2)

In [99]:
predictions

array([[  4.54658538e-01,   2.98151504e-02,   2.04153940e-01,
          5.64452484e-02,   2.10177258e-01,   4.47498634e-02],
       [  4.99796838e-01,   1.71534140e-02,   2.03638375e-01,
          1.33441174e-02,   2.19956189e-01,   4.61110063e-02],
       [  5.07121027e-01,   1.35288248e-02,   2.36039609e-01,
          3.15914187e-03,   2.26269841e-01,   1.38815334e-02],
       ..., 
       [  4.30456221e-01,   3.52740176e-02,   2.15408146e-01,
          4.99149039e-02,   2.31405586e-01,   3.75411101e-02],
       [  6.28165424e-01,   1.01553532e-03,   1.39932096e-01,
          1.42187173e-06,   1.88268512e-01,   4.26170714e-02],
       [  5.21906197e-01,   7.80870626e-03,   2.18505949e-01,
          1.67631975e-03,   2.33591676e-01,   1.65112149e-02]], dtype=float32)

In [100]:
predictions[: , 0]
px = predictions

In [101]:
df = pd.DataFrame({'toxic':px[:,0],'severe_toxic':px[:,1],'obscene':px[:,2] , 'threat':px[:,3] , 'insult':px[:,4] ,'identity_hate':px[:,5]})

In [102]:
df

,identity_hate,insult,obscene,severe_toxic,threat,toxic
0,0.044750,0.210177,0.204154,2.981515e-02,5.644525e-02,0.454659
1,0.046111,0.219956,0.203638,1.715341e-02,1.334412e-02,0.499797
2,0.013882,0.226270,0.236040,1.352882e-02,3.159142e-03,0.507121
3,0.003143,0.193943,0.188158,1.055632e-03,5.926859e-05,0.613642
4,0.010243,0.214843,0.208896,4.666849e-03,4.894359e-03,0.556457
5,0.034504,0.214555,0.205472,1.977843e-02,2.516032e-02,0.500531
6,0.013886,0.223984,0.226706,8.801127e-03,1.677737e-03,0.524944
7,0.022846,0.227198,0.229686,2.135832e-02,9.110350e-03,0.489801
8,0.024413,0.215681,0.212810,8.235474e-03,2.251629e-03,0.536609
9,0.009026,0.215330,0.237528,1.766505e-02,5.619880e-03,0.514831


In [103]:
id_csv = pd.read_csv('test.csv')

In [104]:
df['id'] = id_csv['id']

In [105]:
cols = list(df.columns.values)

In [106]:
cols

['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic', 'id']

In [107]:
df = df[['id','toxic', 'severe_toxic','obscene', 'threat' , 'insult' ,'identity_hate']]

In [108]:
df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.454659,0.029815,0.204154,0.056445,0.210177,0.044750
1,6102620,0.499797,0.017153,0.203638,0.013344,0.219956,0.046111
2,14563293,0.507121,0.013529,0.236040,0.003159,0.226270,0.013882
3,21086297,0.613642,0.001056,0.188158,0.000059,0.193943,0.003143
4,22982444,0.556457,0.004667,0.208896,0.004894,0.214843,0.010243


In [109]:
df.set_index("id").to_csv('Predictions.csv')